### Capstone Project: Melbourne Housing Market

## Load moduls

In [22]:
# data modules
import numpy as np
import scipy.stats as stats
import pandas as pd

# plotting modules
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

# make sure charts appear in the notebook:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

## Load data

In [23]:
df = pd.read_csv('data/houses.csv')

In [24]:
df.price = df.price.str.replace('$','')

In [25]:
df.price = df.price.str.replace(',', '')

In [26]:
def n(x):
    if x == 'Price Withheld':
        return np.nan
    else:
        return x

df.price = df.price.map(n)

In [27]:
df.shape

(393903, 12)

In [28]:
df.dropna(inplace=True)
df.shape

(321844, 12)

In [31]:
def a(x):
    l = x.split(' ')
    date = l[-3:]
    return ' '.join(date)
    
df.solddate = df.solddate.map(a)

In [40]:
def s(x):
    l = x.split(' ')
    return ' '.join(l[:-2])
df['suburb'] = df.address2.map(s)
df.drop('address2', axis=1, inplace=True)

In [50]:
df.bedrooms = df.bedrooms.str.lstrip()

In [53]:
def b(x):
    return x.split(' ')[0]

df.bedrooms =df.bedrooms.map(b)

In [55]:
df.bathrooms = df.bathrooms.str.lstrip()
df.bathrooms =df.bathrooms.map(b)

In [56]:
df.head(2)

,solddate,price,address1,postcode,latitude,longitude,bedrooms,bathrooms,carparkings,agentbrand,location,suburb
0,08 May 2017,165000,907/268 Flinders Street,3000.0,-37.8177643,144.965652,0,1,0 parkings,Logo for Inner Real Estate NEXTRE,"('-37.8177643', '144.965652')",MELBOURNE
1,11 Mar 2017,187700,7/9-13 Anthony Street,3000.0,-37.80922,144.958939,0,1,0 parkings,Logo for Purplebricks VIC,"('-37.80922', '144.958939')",MELBOURNE


In [57]:
df.carparkings = df.carparkings.str.lstrip()
df.carparkings =df.carparkings.map(b)

In [59]:
df.head()

,solddate,price,address1,postcode,latitude,longitude,bedrooms,bathrooms,carparkings,agentbrand,location,suburb
0,08 May 2017,165000,907/268 Flinders Street,3000.0,-37.8177643,144.965652,0,1,0,Logo for Inner Real Estate NEXTRE,"('-37.8177643', '144.965652')",MELBOURNE
1,11 Mar 2017,187700,7/9-13 Anthony Street,3000.0,-37.80922,144.958939,0,1,0,Logo for Purplebricks VIC,"('-37.80922', '144.958939')",MELBOURNE
2,08 Mar 2017,260000,2411/288 Spencer Street,3000.0,-37.8138275,144.952026,0,1,0,Logo for Motion Property,"('-37.8138275', '144.952026')",MELBOURNE
3,01 Dec 2016,162500,609/268 Flinders Street,3000.0,-37.8177643,144.965652,0,1,0,Logo for Inner Real Estate NEXTRE,"('-37.8177643', '144.965652')",MELBOURNE
4,10 Sep 2016,162500,610/268 Flinders Street,3000.0,-37.8177757,144.965546,0,1,0,Logo for Inner Real Estate NEXTRE,"('-37.8177757', '144.965546')",MELBOURNE


In [61]:
def ag(x):
    l = x.split(' ')
    return ' '.join(l[2:])
df.agentbrand = df.agentbrand.map(ag)

In [62]:
df.head(2)

,solddate,price,address1,postcode,latitude,longitude,bedrooms,bathrooms,carparkings,agentbrand,location,suburb
0,08 May 2017,165000,907/268 Flinders Street,3000.0,-37.8177643,144.965652,0,1,0,Inner Real Estate NEXTRE,"('-37.8177643', '144.965652')",MELBOURNE
1,11 Mar 2017,187700,7/9-13 Anthony Street,3000.0,-37.80922,144.958939,0,1,0,Purplebricks VIC,"('-37.80922', '144.958939')",MELBOURNE


In [63]:
df.to_csv('h.csv', index=False)